In [5]:
import requests
import os
import io
import json
from datetime import datetime
from confluent_kafka import Producer, Consumer, KafkaException
from minio import Minio
import uuid

def autenticarOlhoVivo():
    apikey = "{inserir seu token aqui}" #"876r590t4bed4dccbjnnaj9dbebfhbhvbdjndjjdnfj" #INSERIR TOKEN CRIADO
    url = 'https://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token=' + apikey

    payload = {}
    headers = {}

    response = requests.request("POST", url, headers=headers, data=payload)
    get_cookie = response.cookies
    cookie = get_cookie.get("apiCredentials")
    print("Autenticando olhovivo")
    return cookie

def linhasBuscar(cookie):
    url = "https://api.olhovivo.sptrans.com.br/v2.1/Linha/Buscar?termosBusca=8030"
    payload = {}
    headers = {'Cookie': 'apiCredentials=' + cookie}
    response = requests.request("GET", url, headers=headers)
    print("Buscando linhas")
    return response.json()

def ingestaoKafka(data):
    print("Ingestão kafka")
    broker = 'kafka-broker:9092'
    topic = 'api.user'
    producer = Producer({
    'bootstrap.servers' : broker,
    })
    producer.produce(topic, json.dumps(data))
    producer.flush()
    print(f"Message sent to topic '{topic}': {json.dumps(data)}")
    
def workflow():
    cookie = autenticarOlhoVivo()
    data = linhasBuscar(cookie)  
    ingestaoKafka(data)

workflow()

Autenticando olhovivo
Buscando linhas
Ingestão kafka
Message sent to topic 'api.user': [{"cl": 2562, "lc": false, "lt": "8030", "sl": 1, "tl": 10, "tp": "TERM. VL. S\u00d4NIA", "ts": "PARAIS\u00d3POLIS"}, {"cl": 35330, "lc": false, "lt": "8030", "sl": 2, "tl": 10, "tp": "TERM. VL. S\u00d4NIA", "ts": "PARAIS\u00d3POLIS"}]
